<a href="https://colab.research.google.com/github/edufsi/Projeto-Desafio/blob/Eduardo/Limpeza_Dados_Parceiros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [144]:
import pandas as pd
import numpy as np

In [145]:
#Importar dados:
url_commodities = "https://raw.githubusercontent.com/edufsi/Projeto-Desafio/main/Dados/UNdata_Export_20240412_201846138.csv"
url_parceiros = "https://raw.githubusercontent.com/edufsi/Projeto-Desafio/main/Dados/SYB66_330_202310_Major%20Trading%20Partners.csv"
url_exportacoes = "https://raw.githubusercontent.com/edufsi/Projeto-Desafio/main/Dados/SYB66_123_202310_Total%20Imports%20Exports%20and%20Balance%20of%20Trade.csv"
url_cambio = "https://raw.githubusercontent.com/edufsi/Projeto-Desafio/main/Dados/USD_BRL_Historical_Data.csv"


dados_commodities = pd.read_csv(url_commodities)
dados_parceiros = pd.read_csv(url_parceiros, encoding = "ISO-8859-1")
dados_exportacoes = pd.read_csv(url_exportacoes, encoding = "ISO-8859-1")
dados_cambio = pd.read_csv(url_cambio)

In [ ]:
# VISÃO GERAL
dados_parceiros.head()

In [ ]:
dados_parceiros.info()

In [ ]:
# Os titulos das colunas estao errados. Eles estao na segunda coluna. Na primeira temos um titulo
# Podemos usar iloc para selecionar linhas. Passando um intervalo, uma lista ou um numero retornamos as linhas correspondentes


# Vamos tomar a primeira linha e defini-la como nossas colunas
dados_parceiros.columns = dados_parceiros.iloc[0]


# Agora que os titulos ja estao corretos, podemos excluir essa primeira linha que continha os títulos
dados_parceiros.drop(0, inplace=True)


dados_parceiros.info()

In [149]:
# Vamos retirar as colunas Major trading partner 1 (% of exports) footnote e Footnotes, que só dizem se os dados são estimativas ou não
dados_parceiros.drop(["Footnotes", "Major trading partner 1 (% of exports) footnote", "Source"], axis=1, inplace=True)

In [150]:
# No arquivo original, foram associados valores para representar as regioes/paises/areas e há uma coluna sem titulo para a descricao
# Seria melhor aqui ter apenas uma coluna com os nomes das regiões, áreas e dos países. Temos que alterAR entao o nome da segunda coluna, essa sem titulo


lista = [x for x in dados_parceiros.columns]
lista[1] = "Region/Country" # Substituindo o nome da coluna com nan por "Region/Country" (segunda coluna)

dados_parceiros.columns = lista



# Agora podemos retirar a coluna com os códigos
dados_parceiros.drop("Region/Country/Area", axis=1, inplace=True)

In [151]:
# A coluna Value esta como objeto, mas ela deveria ser um valor em porcentagem. Vamos tranforma-la em numero e converte-la para porcentagem

# Tranforma a coluna em numeros de ponto flutuante
dados_parceiros['Value'] = dados_parceiros['Value'].astype(np.float32)


# Os números são porcentagens. Para facilitar contas, vamos transformá-los em decimais
dados_parceiros['Value'] = dados_parceiros['Value'] / 100


In [152]:
# O nome da coluna "Major trading partner 1 (% of exports)" também não faz muito sentido, pois temos linhas que representam o segundo e o terceiro
# maior parceiro. O 1 não faz sentido, nem o indicativo de porcentagem. Vamos trocar o nome dessa coluna e da coluna do valor, especificando que ele é em porcentagem

dados_parceiros = dados_parceiros.rename(columns={"Major trading partner 1 (% of exports)" : "Trading Partner", "Value" : "Value (%)"})


In [153]:
# As informações ficariam ainda mais claras se dividíssemos o dataframe por importações e exportações. Vamos fazer isso

mascara = [True if 'exports' in x else False for x in dados_parceiros['Series']]
exp_parceiros = dados_parceiros[mascara]

mascara = [True if 'imports' in x else False for x in dados_parceiros['Series']]
imp_parceiros = dados_parceiros[mascara]

In [154]:
dados_parceiros

,Region/Country,Year,Series,Trading Partner,Value (%)
1,Afghanistan,2010,Major trading partner 1 (% of exports),Pakistan,0.390
2,Afghanistan,2015,Major trading partner 1 (% of exports),Pakistan,0.397
3,Afghanistan,2022,Major trading partner 1 (% of exports),Pakistan,0.346
4,Afghanistan,2010,Major trading partner 1 (% of imports),United Arab Emirates,0.020
5,Afghanistan,2015,Major trading partner 1 (% of imports),United Arab Emirates,0.042
...,...,...,...,...,...
3686,Zimbabwe,2015,Major trading partner 3 (% of exports),Mozambique,0.120
3687,Zimbabwe,2022,Major trading partner 3 (% of exports),Mozambique,0.084
3688,Zimbabwe,2010,Major trading partner 3 (% of imports),China,0.055
3689,Zimbabwe,2015,Major trading partner 3 (% of imports),China,0.077


In [155]:
exp_parceiros

,Region/Country,Year,Series,Trading Partner,Value (%)
1,Afghanistan,2010,Major trading partner 1 (% of exports),Pakistan,0.390
2,Afghanistan,2015,Major trading partner 1 (% of exports),Pakistan,0.397
3,Afghanistan,2022,Major trading partner 1 (% of exports),Pakistan,0.346
7,Afghanistan,2010,Major trading partner 2 (% of exports),United Arab Emirates,0.016
8,Afghanistan,2015,Major trading partner 2 (% of exports),United Arab Emirates,0.043
...,...,...,...,...,...
3680,Zimbabwe,2015,Major trading partner 2 (% of exports),United Arab Emirates,0.043
3681,Zimbabwe,2022,Major trading partner 2 (% of exports),United Arab Emirates,0.286
3685,Zimbabwe,2010,Major trading partner 3 (% of exports),Mozambique,0.029
3686,Zimbabwe,2015,Major trading partner 3 (% of exports),Mozambique,0.120


In [156]:
imp_parceiros

,Region/Country,Year,Series,Trading Partner,Value (%)
4,Afghanistan,2010,Major trading partner 1 (% of imports),United Arab Emirates,0.020
5,Afghanistan,2015,Major trading partner 1 (% of imports),United Arab Emirates,0.042
6,Afghanistan,2022,Major trading partner 1 (% of imports),United Arab Emirates,0.262
10,Afghanistan,2010,Major trading partner 2 (% of imports),Kazakhstan,0.040
11,Afghanistan,2015,Major trading partner 2 (% of imports),Kazakhstan,0.055
...,...,...,...,...,...
3683,Zimbabwe,2015,Major trading partner 2 (% of imports),Singapore,0.221
3684,Zimbabwe,2022,Major trading partner 2 (% of imports),Singapore,0.138
3688,Zimbabwe,2010,Major trading partner 3 (% of imports),China,0.055
3689,Zimbabwe,2015,Major trading partner 3 (% of imports),China,0.077
